In [1]:
import os, io
import pandas as pd
import pysam
import pickle
import pybedtools
print(pybedtools.__file__)
pybedtools.helpers.set_tempdir('/data/private/pdutta_new/pybedtools_temp')
from IPython.display import display, HTML
import gzip
import shutil
import os

/shared/software/anaconda/anaconda3/lib/python3.10/site-packages/pybedtools/__init__.py


In [3]:
cancer_type="Brain" 

In [4]:
vcf_folder_path="/data/projects/GDC_Cancer_Wise/Old/{}/Data/TCGA_BAM_VCF/VCF_Patientwise".format(cancer_type)
reference_genome_path="/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa"

In [5]:
non_coding_region="donor"

In [6]:
df= pd.read_csv("/data/private/pdutta_new/non_coding_regions/Unique_{}_RAW_Combined_BED.tsv".format(non_coding_region), sep="\t")
df

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12189,12268,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001671638.2, ENSE00002234944.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12659,12738,+,ENST00000450305.2,ENSE00001758273.2,transcribed_unprocessed_pseudogene
2,chr1,12683,12762,+,ENST00000456328.2,ENSE00003582793.1,processed_transcript
3,chr1,13336,13415,+,ENST00000450305.2,ENSE00001746346.2,transcribed_unprocessed_pseudogene
4,chr1,15756,15835,-,ENST00000488147.1,ENSE00002030414.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
297951,chrY,57211531,57211610,+,ENST00000464205.6_PAR_Y,ENSE00003785071.1,processed_transcript
297952,chrY,57211582,57211661,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00003627301.1, ENSE00003905275.1, ENSE0000...","retained_intron, protein_coding"
297953,chrY,57213164,57213243,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
297954,chrY,57213840,57213919,-,ENST00000507418.6_PAR_Y,ENSE00002046926.1,unprocessed_pseudogene


In [7]:
acceptor_bed = pybedtools.BedTool.from_dataframe(df)

In [8]:
reference_fasta = pysam.FastaFile(reference_genome_path)

In [9]:
def get_germline_vcf_files(root_folder):
    germline_files = []

    # Walk through the directory tree
    for dirpath, dirnames, filenames in os.walk(root_folder):
        # If "logs" is in dirnames, remove it to avoid traversing it
        if 'logs' in dirnames:
            dirnames.remove('logs')

        # Add only the filenames with "Germline.vcf" in the current directory to the germline_files list
        for filename in filenames:
            if "Germline.vcf" in filename:
                germline_files.append(os.path.join(dirpath, filename))
    return germline_files

In [10]:
def vcf_to_dataframe(vcf_path):
    """
    Convert a .vcf.gz file into a pandas DataFrame.

    Parameters:
    - vcf_path (str): path to the .vcf.gz file

    Returns:
    - pd.DataFrame: VCF data as a DataFrame
    """
    df = pd.read_csv(vcf_path, sep=",")
    df["START_POS"] = df["POS"]-1
    df['END_POS'] = df["START_POS"] + df['REF'].str.len()
    df = df[["CHROM", "START_POS",'END_POS', "REF", "ALT"]]
    #display(df)
    return df

In [11]:
files = get_germline_vcf_files(vcf_folder_path)

In [12]:
len(files)

154

In [15]:
intersected_vcf_data = {}
df_statistics  = pd.DataFrame(columns= ["filename", "Patient_ID", 'VCF_instance','VCF_feature','Intersected_instances', 'VCF_column_names' ])
for file_path in files:
    file_name =  file_path.split('/')[-1]
    parts = file_name.split('.')[0].split('_')
    print(parts, file_path, parts)
    df_vcf = vcf_to_dataframe(file_path)
    if df_vcf is None:
        continue
    vcf_bed = pybedtools.BedTool.from_dataframe(df_vcf)
    intersect_vcf_acceptor = acceptor_bed.intersect(vcf_bed, wa=True, wb=True)
    column_list  = df.columns.to_list() + df_vcf.columns.to_list()
    df_intersection = intersect_vcf_acceptor.to_dataframe(names=column_list)
    #print(df_intersection.shape)
    #df_intersection = df_intersection[(df_intersection['REF'].str.len() < 10) & (df_intersection['ALT'].str.len() < 10)]
    print(df_intersection.shape)
    # # print(df_intersection[['REF','ALT']])
    # # input()
    # #print(df_intersection)
    # #print(df_statistics)
    # #input()
    #gdc_tag = parts[-4]
    patient_ID = parts[0]
    #work_flow = parts[2]
    instance =  df_vcf.shape[0]
    feature = df_vcf.shape[1]
    intersected_instance = df_intersection.shape[0]
    feature_list = list(df_vcf.columns)
    print(patient_ID, instance, feature, intersected_instance,  feature_list)
    df_statistics.loc[len(df_statistics)] = [file_name, patient_ID, instance, feature,intersected_instance,  feature_list]
    intersected_vcf_data[patient_ID] = df_intersection

['TCGA-02-2483-01A-01D-1494-08', 'TCGA-02-2483-10A-01D-1494-08', 'Germline'] /data/projects/GDC_Cancer_Wise/Old/Brain/Data/TCGA_BAM_VCF/VCF_Patientwise/TCGA-02-2483/TCGA-02-2483-01A-01D-1494-08_TCGA-02-2483-10A-01D-1494-08_Germline.vcf ['TCGA-02-2483-01A-01D-1494-08', 'TCGA-02-2483-10A-01D-1494-08', 'Germline']
(24849, 12)
TCGA-02-2483-01A-01D-1494-08 4269244 5 24849 ['CHROM', 'START_POS', 'END_POS', 'REF', 'ALT']
['TCGA-CS-5396-01A-02D-1465-02', 'TCGA-CS-5396-10A-01D-1465-02', 'Germline'] /data/projects/GDC_Cancer_Wise/Old/Brain/Data/TCGA_BAM_VCF/VCF_Patientwise/TCGA-CS-5396/TCGA-CS-5396-01A-02D-1465-02_TCGA-CS-5396-10A-01D-1465-02_Germline.vcf ['TCGA-CS-5396-01A-02D-1465-02', 'TCGA-CS-5396-10A-01D-1465-02', 'Germline']
(17093, 12)
TCGA-CS-5396-01A-02D-1465-02 2733430 5 17093 ['CHROM', 'START_POS', 'END_POS', 'REF', 'ALT']
['TCGA-EZ-7264-01A-11D-2024-08', 'TCGA-EZ-7264-10A-01D-2024-08', 'Germline'] /data/projects/GDC_Cancer_Wise/Old/Brain/Data/TCGA_BAM_VCF/VCF_Patientwise/TCGA-EZ-7264

In [16]:
df_statistics

,filename,Patient_ID,VCF_instance,VCF_feature,Intersected_instances,VCF_column_names
0,TCGA-02-2483-01A-01D-1494-08_TCGA-02-2483-10A-...,TCGA-02-2483-01A-01D-1494-08,4269244,5,24849,"[CHROM, START_POS, END_POS, REF, ALT]"
1,TCGA-CS-5396-01A-02D-1465-02_TCGA-CS-5396-10A-...,TCGA-CS-5396-01A-02D-1465-02,2733430,5,17093,"[CHROM, START_POS, END_POS, REF, ALT]"
2,TCGA-EZ-7264-01A-11D-2024-08_TCGA-EZ-7264-10A-...,TCGA-EZ-7264-01A-11D-2024-08,4227806,5,24171,"[CHROM, START_POS, END_POS, REF, ALT]"
3,TCGA-16-1460-01A-01D-0932-09_TCGA-16-1460-10A-...,TCGA-16-1460-01A-01D-0932-09,4367829,5,23953,"[CHROM, START_POS, END_POS, REF, ALT]"
4,TCGA-CS-5395-01A-01D-1468-08_TCGA-CS-5395-10A-...,TCGA-CS-5395-01A-01D-1468-08,5144915,5,30411,"[CHROM, START_POS, END_POS, REF, ALT]"
...,...,...,...,...,...,...
149,TCGA-FG-5965-01B-11D-1893-08_TCGA-FG-5965-10A-...,TCGA-FG-5965-01B-11D-1893-08,4425650,5,25370,"[CHROM, START_POS, END_POS, REF, ALT]"
150,TCGA-FG-5965-02B-11D-A29Q-08_TCGA-FG-5965-10A-...,TCGA-FG-5965-02B-11D-A29Q-08,4471888,5,25675,"[CHROM, START_POS, END_POS, REF, ALT]"
151,TCGA-06-0877-01A-01D-0512-09_TCGA-06-0877-10A-...,TCGA-06-0877-01A-01D-0512-09,4031701,5,23495,"[CHROM, START_POS, END_POS, REF, ALT]"
152,TCGA-14-1034-02B-01D-2280-08_TCGA-14-1034-10A-...,TCGA-14-1034-02B-01D-2280-08,3979781,5,23256,"[CHROM, START_POS, END_POS, REF, ALT]"


In [17]:
output_path = "/data/projects/GDC_Cancer_Wise/New_data/{}/Generated_files/Intersected_Data/Germline/non_coding_regions/{}".format(cancer_type, non_coding_region)
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [18]:
df_statistics.to_csv(output_path+"/VCF_statistics.tsv", sep="\t", index=False)

In [19]:
with open(output_path+"/intersected_vcf_{}_data.pkl".format(non_coding_region), "wb") as file:
    pickle.dump(intersected_vcf_data, file)

In [ ]:
df_statistics

In [ ]:
df_statistics[df_statistics['Patient_ID']=="cc884f4d-e62d-4214-bfaa-81052a02a246"]

In [18]:
df_statistics.groupby('work_flow').size()

work_flow
CaVEMan              412
sanger_raw_pindel    412
dtype: int64

In [ ]:
intersected_vcf_data['ef6c3ecc-0bb6-4035-9c45-57740f5bcaa1_sanger_raw_pindel']

In [19]:
len(intersected_vcf_data)

824

In [ ]:
intersected_vcf_data['9e35e7c1-b8e9-441b-9520-3195359f8e43_sanger_raw_pindel']